In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd
import re

In [0]:

service_credential = dbutils.secrets.get(scope="<scope>",key="<service-credential-key>")

spark.conf.set("fs.azure.account.auth.type.<storage-account>.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.<storage-account>.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.<storage-account>.dfs.core.windows.net", "<application-id>")
spark.conf.set("fs.azure.account.oauth2.client.secret.<storage-account>.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.<storage-account>.dfs.core.windows.net", "https://login.microsoftonline.com/<directory-id>/oauth2/token")

In [0]:
#load data to the dataframe
movies_df = spark.read.format('csv')\
        .option("header", True )\
        .option("inferSchema", True )\
        .load('abfss://silver@movierecommendation2025.dfs.core.windows.net/movies/movielens')
ratings_df = spark.read.format('csv')\
        .option("header", True )\
        .option("inferSchema", True )\
        .load('abfss://silver@movierecommendation2025.dfs.core.windows.net/ratings/ratings')

In [0]:
import requests
# TMDB API Key
TMDB_API_KEY = "5f3489021cab8e5617fc32456726cf91"
TMDB_BASE_URL = "https://api.themoviedb.org/3/search/movie"

# Define a function to fetch movie metadata
def fetch_metadata(title):
    params = {
        "api_key": TMDB_API_KEY,
        "query": title
    }
    try:
        response = requests.get(TMDB_BASE_URL, params=params)
        if response.status_code == 200:
            results = response.json().get("results", [])
            if results:
                metadata = results[0]
                return (metadata.get("overview", ""),
                        metadata.get("popularity", 0.0),
                        metadata.get("release_date", ""),
                        metadata.get("vote_average", 0.0),
                        metadata.get("vote_count", 0))
        return ("", 0.0, "", 0.0, 0)
    except Exception as e:
        return ("", 0.0, "", 0.0, 0)

# Register the function as a UDF
fetch_metadata_udf = udf(fetch_metadata, StringType())

# Apply the UDF to the movies DataFrame
movies_enriched_df = movies_df.withColumn("metadata", fetch_metadata_udf(movies_df["title"]))


In [0]:
# Split the metadata into separate columns
movies_enriched_df = movies_enriched_df.withColumn("overview", split(movies_enriched_df["metadata"], ",")[0]) \
                                       .withColumn("popularity", split(movies_enriched_df["metadata"], ",")[1].cast(FloatType())) \
                                       .withColumn("release_date", split(movies_enriched_df["metadata"], ",")[2]) \
                                       .withColumn("vote_average", split(movies_enriched_df["metadata"], ",")[3].cast(FloatType())) \
                                       .withColumn("vote_count", split(movies_enriched_df["metadata"], ",")[4].cast(FloatType())) \
                                       .drop("metadata")

# Show enriched DataFrame
movies_enriched_df.show(5)


+-------+--------------------+--------------------+----+--------------------+----------+------------+------------+----------+
|movieId|               title|              genres|year|            overview|popularity|release_date|vote_average|vote_count|
+-------+--------------------+--------------------+----+--------------------+----------+------------+------------+----------+
|    109|Headless Body in ...|Comedy|Drama|Thri...|1995|[Ljava.lang.Objec...|      NULL|        NULL|        NULL|      NULL|
|    381|When a Man Loves ...|       Drama|Romance|1994|[Ljava.lang.Objec...|      NULL|        NULL|        NULL|      NULL|
|    681|     Coup de torchon|               Crime|1981|[Ljava.lang.Objec...|      NULL|        NULL|        NULL|      NULL|
|    745|Wallace & Gromit:...|Animation|Childre...|1995|[Ljava.lang.Objec...|      NULL|        NULL|        NULL|      NULL|
|   1161|        The Tin Drum|           Drama|War|1979|[Ljava.lang.Objec...|      NULL|        NULL|        NULL|    

In [0]:
movies_enriched_df.write.format('csv')\
    .mode('append')\
    .option("header", "true")\
    .option("path","abfss://gold@movierecommendation2025.dfs.core.windows.net/enrichedMovie")\
    .save()

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:730)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:448)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:448)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecutio